
## Classification

### Load Libraries and Data


In [ ]:
from mne import find_events
from utils.loader import load_data, unicorn_fs, unicorn_channels, convert_to_mne
from utils.preprocessing import extract_epochs, extract_events
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import numpy as np

from utils.validation import plot_confusion_matrix, plot_cross_validated_confusion_matrix

eeg, trigger = load_data("../data/aep/auditory_erp_eyes_open_S1.csv", header=False, fs=unicorn_fs, skiprows=5)
print("Loaded data with shape:" + str(eeg.shape) + " and trigger shape: " + str(trigger.shape))
print("That means we have " + str(eeg.shape[0]) + " samples and " + str(eeg.shape[1]) + " channels.")

 # Convert to MNE format
raw_data = convert_to_mne(eeg, trigger, fs=unicorn_fs, chs=unicorn_channels, recompute=False) # recompute=True to recalculate the event labels if the values are negative

### Data Preparation


1. Extract events and event ids  
2. Epoching and baseline correction
3. Resample and split the data


#### 1.Extract events and event ids

In [ ]:
ev_ids = {'NT': 2, 'T':1}
event_colors = {2:'r', 1:'g'}
stim_channel = 'STI'
events = find_events(raw_data, stim_channel=stim_channel)
raw_data.plot(events=events, event_id=ev_ids, event_color=event_colors, color = 'Gray', block = True, clipping=None, scalings=25e-6)
plt.show()

#### 2.Epoching with baseline correction
The baseline segment is defined as the interval before the onset of the event. The mean of this interval is subtracted from the entire epoch to correct for the baseline shift.

In [ ]:
baseline=(-.6, 0) 
eps = extract_epochs(data=raw_data, events=events, ev_ids=ev_ids, tmin=-0.6, tmax=0.8, baseline=baseline)
eps.apply_baseline(baseline)

X = eps.get_data(picks='eeg')[:, :, 150:350]

y = eps.events[:,-1]

print(X.shape, y.shape)

#### 3.Resample and split the data
There main reason for resampling the data is to avoid unbalanced classes. The number of samples in the minority class is much lower than the majority class. 
This can lead to a model that is biased towards the majority class. 
To avoid this, we can oversample the minority class or undersample the majority class (or both!).

In [ ]:
random_state = 32

# Oversampling the minority class and undersampling the majority class
oversampler = RandomOverSampler(sampling_strategy='minority', random_state=random_state)
undersampler = RandomUnderSampler(sampling_strategy=0.4, random_state=random_state)



# Assuming you have epochs of size (1, 8, 200)
num_epochs = len(X)
epoch_size = (1, 8, 200)

# Reshape the data for LDA
X_r = X.reshape(num_epochs, -1)
y_r = y
X_r, y_r = oversampler.fit_resample(X_r, y) # Uncomment this line to oversample the minority class
#X_r, y_r = undersampler.fit_resample(X_r, y) # Uncomment this line to undersample the majority class

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_r, y_r, test_size=0.2, random_state=33)
print(X_train.shape, X_test.shape)

### Classification
1. Linear Discriminant Analysis
2. Cross-Validation

#### 1.LinearDiscriminantAnalysis
For classification, we use LDA, a model that fits a Gaussian density for each class and has been shown to be a good classifier for ERP data in previous studies [1].
Other classifiers can be used, such as SVM, or variants of LDA such as QDA.
[1] D. J. Krusienski et al., “A comparison of classification techniques for the P300 Speller,” J Neural Eng, vol. 3, no. 4, pp. 299–305, Dec. 2006, doi: 10.1088/1741-2560/3/4/007.


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Apply LDA
model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='binary')

print(f'Accuracy: {accuracy:.2f} F1 Score: {f1:.2f}')
plot_confusion_matrix(y_test, y_pred, classes=['Target', 'Non-Target'], normalize=True)

#### 2.Cross-Validation

In [ ]:
# Use cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(model, X_r, y_r, cv=cv)
f1 = cross_val_score(model, X_r, y_r, cv=cv, scoring='f1', error_score=0)


# Print cross-validated accuracy
print(f'Cross-validated accuracy: {np.mean(scores):.2f} (+/- {np.std(scores):.2f}) F1-Score: {np.mean(f1):.2f}')

plot_cross_validated_confusion_matrix(X_r, y_r, model, cv=cv, classes=['Target', 'Non-Target'], normalize=True)